In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

import pandas as pd

ROOT_PATH = 'C:/AgodaGit/passivbot'
DOWNLOADED_CONFIG_PATH = ROOT_PATH + '/downloaded_configs/passivbot_v5.9.x'
BACKTESTS_PATH = ROOT_PATH + '/backtests/binance'

In [30]:
import glob
import os

# list all json files in cfgs_bkup and its child folders
live_configs = glob.glob(ROOT_PATH + '/configs/live/older_versions/**/*.json', recursive=True)
# normalize json_files by replacing all \\ with /
live_configs = [f.replace('\\', '/') for f in live_configs]


#group json_files by immediate parent folder
currently_running_configs = []
for f in live_configs:
    folder = os.path.dirname(f).split('/')[-1]
    # folder looks like this : 5_9_0 or 5_9_3 or 5_9_4 etc. convert it to v5.9.0 or v.5.9.3 or v5.9.4
    version = 'v' + folder.replace('_', '.')
    currently_running_configs.append({
        'version': version,
        'production_config': f,
        'coin': os.path.basename(f).replace('.json', '')
    })

currently_running_configs

[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/CHZ.json',
  'coin': 'CHZ'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DOGE.json',
  'coin': 'DOGE'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DOT.json',
  'coin': 'DOT'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DYDX.json',
  'coin': 'DYDX'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/EOS.json',
  'coin': 'EOS'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/HBAR.json',
  'coin': 'HBAR'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot

In [27]:
import re
'''
Result file looks like this:
    +------------------------------------------+
    |                 Summary                  |
    +-----------------------------+------------+
    | Metric                      | Value      |
    +-----------------------------+------------+
    | Exchange                    | bybit      |
    | Market type                 | futures    |
    | Symbol                      | ADAUSDT    |
    | Passivbot mode              | emas       |
    | No. days                    | 1100.0     |
    +-----------------------------+------------+
Write a parser function which takes in the path to result.txt file, reads it and then split the Metric and Value columns and put them in a dictionary and return it
'''
def parse_result_file(result_file):
    with open(result_file, 'r') as file:

        pattern = r'/v(\d+\.\d+(?:\.\w+)?)/'
        match = re.search(pattern, result_file)
        if match:
            version = match.group(1)
        else:
            version = None


        data = file.read().split('\n')
        d = {'version':version}
        current_state = None
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()
                # dont add if key is empty string
                if key == '':
                    continue

                if key == 'Long':
                    current_state = 'L'
                if key == 'Short':
                    current_state = 'S'

                # check if value can be converted to float and if so, convert it
                try:
                    value = value.replace('%', '').strip()
                    value = float(value)
                except:
                    pass

                # if current_state is not None, then add it to key with a prefix of (L) or (S)
                if current_state is not None:
                    key = f'({current_state}){key}'
                d[key] = value
    return d

# test it
parse_result_file(r'C:\AgodaGit\passivbot\backtests\binance\ATOMUSDT\plots\2023-08-20T181841\backtest_result.txt')

{'version': None,
 'Metric': 'Value',
 'Exchange': 'binance',
 'Market type': 'futures',
 'Symbol': 'ATOMUSDT',
 'Passivbot mode': 'recursive_grid',
 'ADG n subdivisions': 10.0,
 'No. days': 110.0,
 'Starting balance': 1000.0,
 '(L)Long': 'True',
 '(L)ADG per exposure': 0.0762,
 '(L)ADG weighted per exposure': 0.16,
 '(L)Final balance': 1021.15,
 '(L)Final equity': 1020.23,
 '(L)Net PNL + fees': 21.1527,
 '(L)Net Total gain': 2.115,
 '(L)Average daily gain': 0.019,
 '(L)Average daily gain weighted': 0.04,
 '(L)Loss to profit ratio': 0.4811,
 '(L)Exposure ratios mean': 0.12766,
 '(L)Price action distance mean': 0.0117069,
 '(L)Price action distance std': 0.0186726,
 '(L)Price action distance max': 0.231277,
 '(L)Closest bankruptcy': 100.0,
 '(L)Lowest equity/balance ratio': 0.9521,
 '(L)Mean of 10 worst eq/bal ratios': 0.9533,
 '(L)Equity/balance ratio std': 0.004001,
 '(L)Ratio of time spent at max exposure': 0.07348,
 '(L)No. fills': 660.0,
 '(L)No. entries': 256.0,
 '(L)No. closes': 

In [32]:
import json

# backtest configurations
backtest_configuration = glob.glob(BACKTESTS_PATH + '/**/live_config.json', recursive=True)
# normalize json_files by replacing all \\ with /
backtest_configuration = [f.replace('\\', '/') for f in backtest_configuration]


def compare_key_value_of_two_json_objects(json1, json2):
    
    keys_to_ignore = ['backwards_tp','ema_dist_close','ema_dist_entry']
    
    for key in json1:
        if key in keys_to_ignore:
            continue
        if key not in json2:
            print(f'key {key} not found in json2')
            return False
        if json1[key] != json2[key]:
            print(f'key {key} has different value in json2')
            return False
    return True


# # now iterate over config_by_version and find the downloaded config.json file for each live config
# for version in currently_running_configs:
#     for live_config in currently_running_configs[version]['live_configs']:
#         ''''''

for live_config in currently_running_configs:
    version = live_config['version']
    coin = live_config['coin'] + 'USDT'
    production_config_json = json.load(open(live_config['production_config']))
    # find the list of matches in downloaded config.json files for this coin where version must be in the path and coin must be in the path
    matches = [f for f in backtest_configuration if coin in f]

    match_found = None

    for match in matches:
        live_config_json = json.load(open(match))

        # the live_config_json["long"] and live_config_json["short"] matches file_content['long'] and file_content['short'] respectively, then we have a match
        if compare_key_value_of_two_json_objects(live_config_json["long"], production_config_json['long']) and compare_key_value_of_two_json_objects(
                live_config_json["short"], production_config_json['short']):
            match_found = match
            break

    if match_found is None:
        print(f'No match found for {coin} in {version}')
    else:
        live_config['backtest_config'] = match_found
        # find the backtest result file ( called result.txt ) for this downloaded config.json file. it is found in a subdirectory within the same folder as config.json
        #print(glob.glob(os.path.dirname(match_found)))
        result_file = glob.glob(os.path.dirname(match_found) + '/**/backtest_result.txt', recursive=True)
        live_config['result_file'] = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
        live_config['result'] = parse_result_file(live_config['result_file']) if live_config[
                                                                                     'result_file'] is not None else None
        live_config['result']['version'] = version

currently_running_configs

[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM',
  'backtest_config': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/live_config.json',
  'result_file': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/backtest_result.txt',
  'result': {'version': 'v5.9.0',
   'Metric': 'Value',
   'Exchange': 'binance',
   'Market type': 'futures',
   'Symbol': 'ATOMUSDT',
   'Passivbot mode': 'recursive_grid',
   'ADG n subdivisions': 10.0,
   'No. days': 110.0,
   'Starting balance': 1000.0,
   '(L)Long': 'True',
   '(L)ADG per exposure': 0.0762,
   '(L)ADG weighted per exposure': 0.16,
   '(L)Final balance': 1021.15,
   '(L)Final equity': 1020.23,
   '(L)Net PNL + fees': 21.1527,
   '(L)Net Total gain': 2.115,
   '(L)Average daily gain': 0.019,
   '(L)Average daily gain weighted': 0.04,
   '(L)Loss to profit ratio': 0.4811,
   '(L)Exposure ratios mean': 0.12766,
  

In [34]:
'''
Now iterate over currently_running_configs and add the result's key/value to a dataframe
[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM',
  'backtest_config': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/live_config.json',
  'result_file': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/backtest_result.txt',
  'result': {'version': 'v5.9.0',... }}, 
 ]  
'''

currently_running_df = pd.DataFrame()
for live_config in currently_running_configs:
    if live_config['result'] is None:
        continue
    # add the result's key/value to a dataframe
    for k, v in live_config['result'].items():
        currently_running_df.loc[live_config['coin'], k] = v

# drop Metric , Exchange , Market type
currently_running_df = currently_running_df.drop(['Metric', 'Exchange', 'Market type'], axis=1)

currently_running_df



,version,Symbol,Passivbot mode,ADG n subdivisions,No. days,Starting balance,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Final balance,...,(S)No. unstuck/EMA closes,(S)No. normal closes,(S)Average n fills per day,(S)Mean hours stuck,(S)Max hours stuck,(S)PNL sum,(S)Profit sum,(S)Loss sum,(S)Fee sum,(S)Volume quote
ATOM,v5.9.0,ATOMUSDT,recursive_grid,10.0,110.0,1000.0,True,0.076200,0.16000,1021.150,...,0.0,156.0,2.70,8.78468,194.9500,10.060,10.060,0.0000,-0.5757,2857.00
CHZ,v5.9.0,CHZUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.216000,-0.37300,942.298,...,21.0,279.0,4.74,5.03494,194.9000,14.400,19.130,-4.7360,-1.0340,9742.38
DOGE,v5.9.0,DOGEUSDT,recursive_grid,10.0,110.0,1000.0,True,0.093300,-0.49500,1025.970,...,8.0,112.0,2.07,11.57300,316.3830,19.260,19.480,-0.2149,-0.6449,5114.25
DOT,v5.9.0,DOTUSDT,recursive_grid,10.0,110.0,1000.0,True,0.090900,0.17300,1025.300,...,34.0,231.0,4.72,5.06982,114.4170,1.534,10.970,-9.4380,-0.8145,8942.38
DYDX,v5.9.0,DYDXUSDT,recursive_grid,10.0,110.0,1000.0,True,0.010000,0.08480,1002.760,...,366.0,603.0,14.70,1.63006,61.9167,13.960,69.060,-55.1000,-3.2610,91297.80
EOS,v5.9.0,EOSUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.100000,0.14600,972.835,...,35.0,234.0,4.73,5.03118,195.5500,14.150,23.490,-9.3460,-1.3170,14302.20
HBAR,v5.9.0,HBARUSDT,recursive_grid,10.0,110.0,1000.0,True,0.093800,0.53600,1026.120,...,27.0,322.0,6.10,3.93149,157.1670,17.560,28.400,-10.8400,-1.9340,17058.40
LINK,v5.9.0,LINKUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.006150,0.00356,998.311,...,7.0,137.0,2.46,9.71499,168.1500,8.274,9.798,-1.5240,-0.4496,4103.77
MANA,v5.9.0,MANAUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.037400,0.08570,989.783,...,29.0,181.0,3.44,6.92095,184.3330,17.170,23.470,-6.3010,-0.6768,10256.40
MATIC,v5.9.0,MATICUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.191000,-0.41800,948.924,...,6.0,256.0,4.79,4.94106,231.4500,16.810,19.470,-2.6550,-0.8474,5051.55


In [38]:
def shorten_string(string):
    # replace everything between parentheses
    string = re.sub(r'\([^)]*\)', '', string)

    # create a dictionary of key value to replace in string
    replace_dict = {'ADG per exposure': 'adg_per_exp',
                    'ADG weighted per exposure': 'adg_w_per_exp',
                    'Average daily gain': 'adg',
                    'Average daily gain weighted': 'adg_w',
                    'Closest bankruptcy': 'bnkr',
                    'Equity/balance ratio std': 'ebr_std',
                    'Exposure ratios mean': 'exp_rat_mean',
                    'Loss to profit ratio': 'l2p_r',
                    'Lowest equity/balance ratio': 'l_ebr',
                    'Max hours stuck': 'max_stuck',
                    'Mean hours stuck': 'mean_stuck',
                    'Mean of 10 worst eq/bal ratios': 'mean_10_ebr',
                    'No. initial entries': 'n_init_ent',
                    'No. normal closes': 'n_norm_close',
                    'No. reentries': 'n_re_ent',
                    'No. unstuck closes': 'n_unstk_close',
                    'No. unstuck entries': 'n_unstk_ent',
                    'Price action distance max': 'pa_d_max',
                    'Price action distance mean': 'pa_d_mean',
                    'Price action distance std': 'pa_d_std',
                    'ADG n subdivisions': 'adg_n_subdiv',
                    'Starting balance':'start_bal',
                    'Final balance':'final_bal',
                    'No. unstuck/EMA closes':'n_unstk_ema_close',
                    'Average n fills per day':'avg_n_fills_per_day',
                    'PNL sum':'pnl_sum',
                    'Profit sum':'profit_sum',
                    'Loss sum':'loss_sum',
                    'Fee sum'  :'fee_sum',
                    'Volume quote':'vol_quote',
                    'Exchange': 'exch',
                    'No. days': 'n_days',
                    'Passivbot mode': 'mode',
                    'Symbol': 's',
                    'file': 'f',
                    'live': 'live',
                    'min_notional': 'min_notion',
                    'version': 'ver',
                    'volume': 'vol'}

    # find the longest key match in replace_dict and replace it with its value
    for k in sorted(replace_dict, key=len, reverse=True):
        string = string.replace(k, replace_dict[k])

    return string



'''
call shorten_string function with the followings and print result:
 '(L)ADG per exposure'
 '(L)ADG weighted per exposure'
 '(L)Average daily gain'
 '(L)Average daily gain weighted'
'''
print(shorten_string('(L)ADG per exposure'))
print(shorten_string('(L)ADG weighted per exposure'))
print(shorten_string('(L)Average daily gain'))
print(shorten_string('(L)Average daily gain weighted'))

# create currently_running_df_long by dropping all columns which has (S) in it
currently_running_df_long = currently_running_df[[c for c in currently_running_df.columns if '(S)' not in c]]
# create currently_running_df_short by dropping all columns which has (L) in it
currently_running_df_short = currently_running_df[[c for c in currently_running_df.columns if '(L)' not in c]]

currently_running_df_long = currently_running_df_long.rename(columns=shorten_string)
currently_running_df_short = currently_running_df_short.rename(columns=shorten_string)

display(currently_running_df_long , currently_running_df_short)

# dump it in an excel for analysis
EXCEL_FILE_NAME = r'C:\AgodaGit\passivbot\analysis_scripts\current_running_configs.xlsx'
excel_writer = pd.ExcelWriter(EXCEL_FILE_NAME, engine='xlsxwriter')
currently_running_df_long.to_excel(excel_writer, index=False, sheet_name='long')
currently_running_df_short.to_excel(excel_writer, index=False, sheet_name='short')
excel_writer.save()
excel_writer.close()


adg_per_exp
adg_w_per_exp
adg
adg_w


,ver,s,mode,adg_n_subdiv,n_days,start_bal,Long,adg_per_exp,adg_w_per_exp,final_bal,...,n_unstk_ema_close,n_norm_close,avg_n_fills_per_day,mean_stuck,max_stuck,pnl_sum,profit_sum,loss_sum,fee_sum,vol_quote
ATOM,v5.9.0,ATOMUSDT,recursive_grid,10.0,110.0,1000.0,True,0.076200,0.16000,1021.150,...,22.0,382.0,6.00,3.98480,101.8500,23.010,44.35,-21.340,-1.8590,24807.20
CHZ,v5.9.0,CHZUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.216000,-0.37300,942.298,...,328.0,201.0,7.98,2.99886,194.1170,-55.980,44.68,-100.700,-1.7190,139194.00
DOGE,v5.9.0,DOGEUSDT,recursive_grid,10.0,110.0,1000.0,True,0.093300,-0.49500,1025.970,...,136.0,595.0,11.10,2.15354,71.0833,29.840,82.54,-52.710,-3.8700,78478.70
DOT,v5.9.0,DOTUSDT,recursive_grid,10.0,110.0,1000.0,True,0.090900,0.17300,1025.300,...,28.0,323.0,5.21,4.57669,170.2670,26.680,37.47,-10.790,-1.3830,25516.50
DYDX,v5.9.0,DYDXUSDT,recursive_grid,10.0,110.0,1000.0,True,0.010000,0.08480,1002.760,...,17.0,111.0,2.28,10.31910,289.7830,3.614,25.68,-22.070,-0.8543,12858.90
EOS,v5.9.0,EOSUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.100000,0.14600,972.835,...,134.0,414.0,8.39,2.84662,167.1670,-25.450,49.53,-74.980,-2.2910,69937.10
HBAR,v5.9.0,HBARUSDT,recursive_grid,10.0,110.0,1000.0,True,0.093800,0.53600,1026.120,...,88.0,532.0,9.76,2.44823,108.1670,28.580,65.47,-36.890,-2.4160,56562.40
LINK,v5.9.0,LINKUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.006150,0.00356,998.311,...,58.0,84.0,2.47,9.70720,184.9670,-1.016,23.89,-24.910,-0.6728,27479.90
MANA,v5.9.0,MANAUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.037400,0.08570,989.783,...,54.0,623.0,9.88,2.35880,123.6670,-6.964,55.73,-62.690,-3.2530,48608.30
MATIC,v5.9.0,MATICUSDT,recursive_grid,10.0,110.0,1000.0,True,-0.191000,-0.41800,948.924,...,222.0,604.0,12.50,1.91808,82.3833,-48.450,63.02,-111.500,-3.5430,106182.00


,ver,s,mode,adg_n_subdiv,n_days,start_bal,Short,adg_per_exp,adg_w_per_exp,final_bal,...,n_unstk_ema_close,n_norm_close,avg_n_fills_per_day,mean_stuck,max_stuck,pnl_sum,profit_sum,loss_sum,fee_sum,vol_quote
ATOM,v5.9.0,ATOMUSDT,recursive_grid,10.0,110.0,1000.0,True,0.07150,0.0603,1009.480,...,0.0,156.0,2.70,8.78468,194.9500,10.060,10.060,0.0000,-0.5757,2857.00
CHZ,v5.9.0,CHZUSDT,recursive_grid,10.0,110.0,1000.0,True,0.10100,0.1230,1013.360,...,21.0,279.0,4.74,5.03494,194.9000,14.400,19.130,-4.7360,-1.0340,9742.38
DOGE,v5.9.0,DOGEUSDT,recursive_grid,10.0,110.0,1000.0,True,0.14000,0.1150,1018.620,...,8.0,112.0,2.07,11.57300,316.3830,19.260,19.480,-0.2149,-0.6449,5114.25
DOT,v5.9.0,DOTUSDT,recursive_grid,10.0,110.0,1000.0,True,0.00545,0.0545,1000.720,...,34.0,231.0,4.72,5.06982,114.4170,1.534,10.970,-9.4380,-0.8145,8942.38
DYDX,v5.9.0,DYDXUSDT,recursive_grid,10.0,110.0,1000.0,True,0.08070,0.1720,1010.700,...,366.0,603.0,14.70,1.63006,61.9167,13.960,69.060,-55.1000,-3.2610,91297.80
EOS,v5.9.0,EOSUSDT,recursive_grid,10.0,110.0,1000.0,True,0.09660,0.1060,1012.830,...,35.0,234.0,4.73,5.03118,195.5500,14.150,23.490,-9.3460,-1.3170,14302.20
HBAR,v5.9.0,HBARUSDT,recursive_grid,10.0,110.0,1000.0,True,0.11800,-0.0828,1015.620,...,27.0,322.0,6.10,3.93149,157.1670,17.560,28.400,-10.8400,-1.9340,17058.40
LINK,v5.9.0,LINKUSDT,recursive_grid,10.0,110.0,1000.0,True,0.05910,0.0862,1007.820,...,7.0,137.0,2.46,9.71499,168.1500,8.274,9.798,-1.5240,-0.4496,4103.77
MANA,v5.9.0,MANAUSDT,recursive_grid,10.0,110.0,1000.0,True,0.12400,0.1130,1016.490,...,29.0,181.0,3.44,6.92095,184.3330,17.170,23.470,-6.3010,-0.6768,10256.40
MATIC,v5.9.0,MATICUSDT,recursive_grid,10.0,110.0,1000.0,True,0.12000,0.0852,1015.970,...,6.0,256.0,4.79,4.94106,231.4500,16.810,19.470,-2.6550,-0.8474,5051.55


C:\Users\skhan\.virtualenvs\passivbot\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
